# Megaline Telecom: Prepaid Plan Revenue Analysis

## Project Goal

As a Data Analyst for Megaline, I need to compare the two main prepaid plans, **Surf** and **Ultimate**, to determine which one generates more revenue for the company. This analysis, based on a sample of 500 customers in 2018, will guide the commercial department in optimizing their advertising budget.

## 1. Initialization

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from math import factorial 
import seaborn as sns
from scipy.stats import ttest_ind

## 2. Data Loading

Loading all five necessary datasets from the local directory.

In [2]:
# Define the absolute file path based on local machine setup
file_path_base = 'C:/Users/Note/Desktop/sprints/sprint 4/'

try:
    # Load datasets
    df_calls = pd.read_csv(file_path_base + 'megaline_calls.csv') 
    df_internet = pd.read_csv(file_path_base + 'megaline_internet.csv') 
    df_messages = pd.read_csv(file_path_base + 'megaline_messages.csv') 
    df_plans = pd.read_csv(file_path_base + 'megaline_plans.csv') 
    df_users = pd.read_csv(file_path_base + 'megaline_users.csv') 
    print("All files loaded successfully.")
except FileNotFoundError:
    print(f"Error: CSV files not found in '{file_path_base}'. Please check your file path.")
    raise

All files loaded successfully.


## 3. Data Preprocessing & Cleaning

### 3.1. Plans Table (`df_plans`)

Inspecting the plan details and converting the internet allowance from MB to GB for consistency with the billing structure (which charges per GB).

In [3]:
# Initial inspection
print(df_plans.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 260.0+ bytes
None


In [4]:
print(df_plans.sample())

   messages_included  mb_per_month_included  minutes_included  \
0                 50                  15360               500   

   usd_monthly_pay  usd_per_gb  usd_per_message  usd_per_minute plan_name  
0               20          10             0.03            0.03      surf  


#### Correcting Data in Plans Table

Since Megaline bills per full GB, I'm converting the MB allowance to GB, rounding up to reflect the unit of measurement used for overage charges.

In [5]:
df_plans['gb_per_month_included'] = np.ceil(df_plans['mb_per_month_included'] /1024).astype(int)

print(df_plans['gb_per_month_included'])

0    15
1    30
Name: gb_per_month_included, dtype: int32


### 3.2. Users Table (`df_users`)

Checking for missing values in `churn_date` and creating a flag for active users.

In [6]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [7]:
print(df_users.sample(15))

     user_id first_name   last_name  age  \
96      1096       Song       Cline   28   
43      1043     Carlos      Hinton   42   
10      1010     Wilber       Blair   52   
193     1193   Lacresha       Olsen   18   
351     1351       Kyle      Grimes   47   
209     1209     Garret        Paul   63   
491     1491     Terina      Jensen   22   
22      1022         Bo        Snow   73   
58      1058   Reinaldo       Colon   59   
222     1222       Cleo    Anderson   18   
473     1473       Kirk       Velez   61   
251     1251     Tifany       Mcgee   45   
163     1163      Lanie  Washington   27   
403     1403        Jae     Gardner   27   
83      1083     Lyndia     Acevedo   65   

                                                 city    reg_date      plan  \
96          New York-Newark-Jersey City, NY-NJ-PA MSA  2018-12-22  ultimate   
43          New York-Newark-Jersey City, NY-NJ-PA MSA  2018-06-20  ultimate   
10                Dallas-Fort Worth-Arlington, TX MSA  201

#### Initial Observations (Users)

The NaNs in `churn_date` indicate users who were still active. I'll create a boolean flag, `is_active`, to easily filter this group later.

#### Correcting Data in Users Table

In [8]:
df_users['is_active'] = df_users['churn_date'].isna()

print(df_users['is_active'])

0      True
1      True
2      True
3      True
4      True
       ... 
495    True
496    True
497    True
498    True
499    True
Name: is_active, Length: 500, dtype: bool


In [9]:
[col.strip().lower() for col in df_users.columns] # Quick check on column names


['user_id',
 'first_name',
 'last_name',
 'age',
 'city',
 'reg_date',
 'plan',
 'churn_date',
 'is_active']

### 3.3. Calls Table (`df_calls`)

The main issue here is applying Megaline's billing rule: call durations must be rounded up to the next minute. I also need to extract the month for monthly usage aggregation.

In [10]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


In [11]:
df_calls.sample(15)

,id,user_id,call_date,duration
106889,1379_87,1379,2018-12-27,2.39
102363,1362_1140,1362,2018-02-23,6.23
93956,1335_186,1335,2018-11-23,15.26
39350,1147_74,1147,2018-12-10,14.93
94416,1336_276,1336,2018-11-02,10.26
35744,1133_425,1133,2018-12-23,6.60
135449,1491_341,1491,2018-10-26,0.00
103042,1365_238,1365,2018-10-29,5.57
101291,1362_26,1362,2018-12-03,4.56
34814,1132_94,1132,2018-04-19,6.05


#### Correcting Data in Calls Table

Applying rounding and converting the `call_date` to extract monthly data.

In [12]:
# Round up call duration to the nearest minute and convert to integer
df_calls['duration'] = np.ceil(df_calls['duration']).astype(int)

# Convert to datetime and extract components
df_calls['call_date'] = pd.to_datetime(df_calls['call_date'])
df_calls['c_year'] = df_calls['call_date'].dt.year
df_calls['c_month'] = df_calls['call_date'].dt.month
df_calls['c_day'] = df_calls['call_date'].dt.day 

print(df_calls.head())

         id  user_id  call_date  duration  c_year  c_month  c_day
0   1000_93     1000 2018-12-27         9    2018       12     27
1  1000_145     1000 2018-12-27        14    2018       12     27
2  1000_247     1000 2018-12-27        15    2018       12     27
3  1000_309     1000 2018-12-28         6    2018       12     28
4  1000_380     1000 2018-12-30         5    2018       12     30


### 3.4. Messages Table (`df_messages`)

Converting the `message_date` to datetime to extract the month for aggregation.

In [13]:
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [14]:
df_messages['message_date'] = pd.to_datetime(df_messages['message_date'])

df_messages['m_year'] = df_messages['message_date'].dt.year
df_messages['m_month'] = df_messages['message_date'].dt.month
df_messages['m_day'] = df_messages['message_date'].dt.day 

df_messages.sample(15)

,id,user_id,message_date,m_year,m_month,m_day
27579,1172_36,1172,2018-09-30,2018,9,30
58223,1369_24,1369,2018-12-13,2018,12,13
43724,1291_27,1291,2018-10-31,2018,10,31
41602,1267_113,1267,2018-12-02,2018,12,2
42570,1280_54,1280,2018-07-20,2018,7,20
50299,1329_46,1329,2018-09-03,2018,9,3
16110,1110_1,1110,2018-08-14,2018,8,14
31649,1196_132,1196,2018-02-23,2018,2,23
45067,1302_172,1302,2018-11-28,2018,11,28
59899,1381_12,1381,2018-10-06,2018,10,6
